# <b>1. library Installation</b>

## <b>1.1 Install library</b>

In [1]:
#install pyspark
! pip install pyspark

#install panel dan hvplot
! pip install panel hvplot
! pip install jupyter_bokeh

## <b>1.2 Mengimport modul dan membuat session</b>


In [2]:
#mengimport modul yang dibutuhkan
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import json

#membuat session
appName = "Analisis Trending Youtube"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# <b>2. Data Collecting</b>


## <b>2.1 Memuat data dari file</b>

In [3]:
#membaca data dari file ke DataFrame dengan skema yang diinfer
trend_csv = spark.read.csv("C:/Users/LENOVO/Desktop/Praktik Coding/Big Data_TA/USvideos.csv", inferSchema=True, header=True)
trend_csv.show(truncate=False)

+-----------+-------------+--------------------------------------------------------------------------------------+---------------------+-----------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------

Jika kita melihat kolom trending_date atau publish_time , kita melihat bahwa mereka belum dalam format data datetime yang benar.

In [4]:
# Menampilkan jumlah baris data
print("Jumlah baris data:", trend_csv.count())

Jumlah baris data: 48137


# <b>3. Data Preprocessing</b>


## <b>3.1 Konversi Format Data Datetime</b>


In [5]:
# Konversi kolom 'trending_date' ke tipe tanggal
trend_csv = trend_csv.withColumn('trending_date', to_date(trend_csv['trending_date'], 'yy.dd.MM'))
# Konversi kolom 'publish_time' ke tipe timestamp
trend_csv = trend_csv.withColumn('publish_time', to_timestamp(trend_csv['publish_time'], 'yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\''))
# Tampilkan beberapa baris pertama dari kolom 'trending_date'
trend_csv.select('video_id', 'trending_date', 'publish_time').show(truncate=False)

+-----------+-------------+-------------------+
|video_id   |trending_date|publish_time       |
+-----------+-------------+-------------------+
|2kyS6SvSYSE|2017-11-14   |2017-11-13 17:13:01|
|1ZAPwfrtAFY|2017-11-14   |2017-11-13 07:30:00|
|5qpjK5DgCt4|2017-11-14   |2017-11-12 19:05:24|
|puqaWrEC7tY|2017-11-14   |2017-11-13 11:00:04|
|d380meD0W0M|2017-11-14   |2017-11-12 18:01:41|
|gHZ1Qz0KiKM|2017-11-14   |2017-11-13 19:07:23|
|39idVpFF7NQ|2017-11-14   |2017-11-12 05:37:17|
|nc99ccSXST0|2017-11-14   |2017-11-12 21:50:37|
|jr9QtXwC9vc|2017-11-14   |2017-11-13 14:00:23|
|TUmyygCMMGA|2017-11-14   |2017-11-13 13:45:16|
|9wRQljFNDW8|2017-11-14   |2017-11-13 02:05:26|
|VifQlJit6A0|2017-11-14   |2017-11-13 03:00:00|
|5E4ZBSInqUU|2017-11-14   |2017-11-13 17:00:00|
|GgVmn66oK_A|2017-11-14   |2017-11-12 14:00:00|
|TaTleo4cOs8|2017-11-14   |2017-11-12 18:30:01|
|kgaO45SyaO4|2017-11-14   |2017-11-13 20:09:58|
|ZAQs-ctOqXQ|2017-11-14   |2017-11-12 17:00:05|
|YVfyYrEmzgM|2017-11-14   |2017-11-13 16

## <b>3.2 Ekstrak Category Id Dan Category Title Pada File JSON</b>


In [6]:
# Baca file JSON
multiline_df = spark.read.option("multiline", "true").json("C:/Users/LENOVO/Desktop/Praktik Coding/Big Data_TA/US_category_id.json")

# Cetak skema JSON
multiline_df.printSchema()

# Memecah nilai array menjadi baris terpisah
exploded_df = multiline_df.select(explode("items").alias("item"))

# Mengakses kolom yang tepat dalam struktur JSON
category_df = exploded_df.select("item.id", "item.snippet.title")
category_df = category_df.withColumnRenamed("title", "category_title")

# Menampilkan hasil
category_df.show()

root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- assignable: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |-- kind: string (nullable = true)

+---+--------------------+
| id|      category_title|
+---+--------------------+
|  1|    Film & Animation|
|  2|    Autos & Vehicles|
| 10|               Music|
| 15|      Pets & Animals|
| 17|              Sports|
| 18|        Short Movies|
| 19|     Travel & Events|
| 20|              Gaming|
| 21|       Videoblogging|
| 22|      People & Blogs|
| 23|              Comedy|
| 24|       Entertainment|
| 25|     News & Politics|
| 26|       Howto & Style|
| 27|           Education|
|

## <b>3.3 Menggabungkan Tabel</b>


In [7]:
# Menggabungkan DataFrame berdasarkan kolom tertentu
joined_df = trend_csv.join(category_df,trend_csv.category_id == category_df.id,"inner")
joined_df.show()

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+---+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description| id|      category_title|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+---+--------------------+
|2kyS6SvSYSE|   2017-11-14|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13 17:13:01|     SHANtell martin| 748374| 57527|    2

## <b>3.4 Melihat Missing Value</b>


In [8]:
# Memeriksa nilai NaN (null) pada DataFrame
nan_counts = joined_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in joined_df.columns])

# Menampilkan jumlah nilai NaN (null) pada setiap kolom
nan_counts.show()

+--------+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+--------------+-----------------+----------------+----------------------+-----------+---+--------------+
|video_id|trending_date|title|channel_title|category_id|publish_time|tags|views|likes|dislikes|comment_count|thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|description| id|category_title|
+--------+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+--------------+-----------------+----------------+----------------------+-----------+---+--------------+
|       0|            0|    0|            0|          0|           0|   0|    0|    0|       0|            0|             0|                0|               0|                     0|        570|  0|             0|
+--------+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+--------------+-----------------+--

In [9]:
# Memfilter baris DataFrame berdasarkan nilai NaN pada kolom 'description'
filtered_df = joined_df.filter(col('description').isNull())

# Menampilkan DataFrame yang sudah difilter
filtered_df.show()

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+-----------+---+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|description| id|      category_title|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+-----------+---+--------------------+
|NZFhMSgbKKM|   2017-11-14|Dennis Smith Jr. ...|        Ben Rohrbach|         17|2017-11-13 15:11:00|              [none]|    945|    7|       5|            8|https://i.yti

kolom 'tags' memiliki string '[none]' karena ini adalah string, python tidak dapat mendeteksinya sebagai nilai NaN 

In [10]:
# Menggantikan nilai '[none]' dengan NaN pada kolom 'tags'
joined_df = joined_df.withColumn('tags', when(col('tags') == '[none]', None).otherwise(col('tags')))

# Memeriksa nilai NaN (null) pada DataFrame
nan_counts = joined_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in joined_df.columns])

# Menampilkan jumlah nilai NaN (null) pada setiap kolom
nan_counts.show()

+--------+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+--------------+-----------------+----------------+----------------------+-----------+---+--------------+
|video_id|trending_date|title|channel_title|category_id|publish_time|tags|views|likes|dislikes|comment_count|thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|description| id|category_title|
+--------+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+--------------+-----------------+----------------+----------------------+-----------+---+--------------+
|       0|            0|    0|            0|          0|           0|1535|    0|    0|       0|            0|             0|                0|               0|                     0|        570|  0|             0|
+--------+-------------+-----+-------------+-----------+------------+----+-----+-----+--------+-------------+--------------+-----------------+--

## <b>3.5 Menyesuaikan Tipe Data</b>


In [11]:
# Menampilkan skema DataFrame
joined_df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: date (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)
 |-- id: string (nullable = true)
 |-- category_title: string (nullable = true)



In [12]:
# Daftar kolom yang ingin diubah tipe datanya menjadi integer
int_columns = ["views", "likes", "dislikes", "comment_count"]

# Mengubah tipe data kolom menjadi integer menggunakan loop
for col_name in int_columns:
    joined_df = joined_df.withColumn(col_name, col(col_name).cast("int"))

# Menampilkan skema DataFrame
joined_df.printSchema()

# Menampilkan DataFrame hasil konversi
joined_df.show()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: date (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)
 |-- id: string (nullable = true)
 |-- category_title: string (nullable = true)

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----

# <b>4. Exploratory Data Analysis</b>


In [13]:
import panel as pn

# Convert to DataFrame
youtube_df = joined_df.toPandas()

# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = youtube_df

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\pandas\conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


## <b>4.1 Membuat Data Interaktif Untuk Pipeline</b>

In [14]:
import pandas as pd
import panel as pn
import hvplot.pandas
import numpy as np
import panel as pn
import hvplot.pandas
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
pn.extension('tabulator')
# %%
# Convert 'trending_date' column to datetime
youtube_df['trending_date'] = pd.to_datetime(youtube_df['trending_date'])
youtube_df['publish_time'] = pd.to_datetime(youtube_df['publish_time'])
youtube_df['views'] = youtube_df['views'].astype(int)
youtube_df['likes'] = youtube_df['likes'].astype(int)
youtube_df['dislikes'] = youtube_df['dislikes'].astype(int)
youtube_df['comment_count'] = youtube_df['comment_count'].astype(int)


# Set 'trending_date' as the index
# youtube_df = youtube_df.set_index('publish_time')

# Make DataFrame Pipeline Interactive
idf = youtube_df.interactive()
idf.dtypes



BokehModel(combine_events=True, render_bundle={'docs_json': {'00fd86db-7150-4ea3-81ca-52298020c58c': {'version…

## <b>4.2 Jumlah Video Trending Berdasarkan Kategori</b>


In [15]:
import datetime

# Define Panel widgets
min_tahun_upload = youtube_df['trending_date'].min().date()
max_tahun_upload = youtube_df['trending_date'].max().date()

range_year_slider = pn.widgets.DateSlider(
    name='Rentang Waktu',
    start=min_tahun_upload,
    end=max_tahun_upload,
    value=(max_tahun_upload),
    bar_color='#D95252'
)

cat_count_pipeline = (
    idf[(idf.trending_date.dt.date <= range_year_slider)]
    .groupby(['category_title']).size()
    .to_frame(name='jumlah_data')
    .reset_index()
    .sort_values(by='jumlah_data')  
    .reset_index(drop=True))

plot_count_cat = cat_count_pipeline.hvplot.barh(y='jumlah_data', x='category_title', color='#D95252', line_width=0, title="Jumlah Video Trending Dari Waktu Ke Waktu Berdasarkan Kategori", xlabel = '', ylabel = 'Jumlah')
plot_count_cat

BokehModel(combine_events=True, render_bundle={'docs_json': {'1a5eab86-a3e6-47af-856c-50b8a2d18779': {'version…

## <b>4.3 Jumlah Metrik Video Trending Dari Waktu Ke Waktu Berdasarkan Kategori</b>


In [ ]:
kategori = list(youtube_df['category_title'].unique())
multi_choice = pn.widgets.MultiSelect(name='Kategori Video', value=['People & Blogs'], size=5,
    options=kategori)

# Radio buttons
yaxis_yutub = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options= {'Views':'views', 'Likes':'likes', 'Dislikes':'dislikes', 'Comment Count':'comment_count'}, 
    button_type='danger'
)

cat_metric_pipeline = (
    idf[
        (idf.trending_date.dt.date <= range_year_slider) &
        (idf.category_title.isin(multi_choice))
    ]
    .groupby(['category_title', 'trending_date'])[yaxis_yutub].mean().round(0)
    .to_frame()
    .reset_index()
    .sort_values(by='trending_date')  
    .reset_index(drop=True)
)

time_line_plot = cat_metric_pipeline.hvplot(x = 'trending_date', by='category_title', y=yaxis_yutub, line_width=2, title='Metrik Video Trending Dari Waktu Ke Waktu Berdasarkan Kategori')
time_line_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'7816fa21-e01a-4407-9a54-4386c1c2fb1e': {'version…

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': '10c42761-2354-41a6-8637-5df9cd6ec8f0'}, 'attr': 'inner_width', 'new': 1021}, {'kind': 'ModelChanged', 'model': {'id': '10c42761-2354-41a6-8637-5df9cd6ec8f0'}, 'attr': 'inner_height', 'new': 210}, {'kind': 'ModelChanged', 'model': {'id': '10c42761-2354-41a6-8637-5df9cd6ec8f0'}, 'attr': 'outer_width', 'new': 1200}, {'kind': 'ModelChanged', 'model': {'id': '10c42761-2354-41a6-8637-5df9cd6ec8f0'}, 'attr': 'outer_height', 'new': 300}]} 
 error: DeserializationError("can't resolve reference '10c42761-2354-41a6-8637-5df9cd6ec8f0'")
Traceback (most recent call last):
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\bokeh\server\sessi

## <b> 4.4 Membuat Dashboard</b>

In [ ]:
import panel as pn


#Layout using Template
template = pn.template.FastListTemplate(
    title='Trending YouTube  Video Dashboard', 
    sidebar=[pn.pane.Markdown("# US Trending YouTube Videos"), 
             pn.pane.PNG("C:/Users/LENOVO/Desktop/Praktik Coding/Big Data_TA/yutub.png", sizing_mode="scale_both"), 
             pn.pane.Markdown("<p align='justify'>US trending YouTube video adalah video-video yang saat ini populer atau mendapatkan popularitas tinggi di platform YouTube di Amerika Serikat. Video-video ini muncul di bagian 'Trending' di halaman beranda YouTube dan dipilih berdasarkan faktor-faktor seperti jumlah View, Likes, Dislike, dan Komentar, interaksi, dan pertumbuhan penonton. Mereka mencakup berbagai genre dan topik, termasuk musik, vlog, video lucu, liputan berita, game, dan lain sebagainya. Melihat daftar trending di US YouTube dapat memberikan wawasan tentang tren dan konten yang sedang populer di kalangan penonton Amerika Serikat."), 
             pn.pane.Markdown("## Pengaturan"),   
             range_year_slider, yaxis_yutub, multi_choice],
    main=[pn.Row(pn.Column(plot_count_cat.panel(width=1200), margin=(0,25))), 
          pn.Row(pn.Column(time_line_plot.panel(width=1200), margin=(0,25)))],
    accent_base_color="#D95252",
    header_background="#D95252",
)
template.show()
#template.servable();

Launching server at http://localhost:59243
